<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


Read data created in the previous chapter.

In [2]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [3]:
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [4]:
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

Our input function for predictions is the same except we don't provide a label

In [6]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

### Create feature columns for estimator

In [7]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [8]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_master': '', '_keep_checkpoint_max': 5, '_session_config': None, '_is_chief': True, '_service': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffa1768bda0>, '_global_id_in_cluster': 0, '_train_distribute': None, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_task_type': 'worker', '_task_id': 0, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_evaluation_master': '', '_log_step_count_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 27100.957, st

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [9]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-22-16:51:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-22-16:51:16
INFO:tensorflow:Saving dict for global step 608: average_loss = 111.87226, global_step = 608, loss = 13304.808
RMSE on dataset = 10.5769681930542


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [10]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([9.882024], dtype=float32)}
{'predictions': array([9.879708], dtype=float32)}
{'predictions': array([9.880491], dtype=float32)}
{'predictions': array([9.878273], dtype=float32)}
{'predictions': array([9.881786], dtype=float32)}
{'predictions': array([9.881588], dtype=float32)}
{'predictions': array([9.880298], dtype=float32)}
{'predictions': array([9.880325], dtype=float32)}
{'predictions': array([9.881923], dtype=float32)}
{'predictions': array([9.879983], dtype=float32)}
{'predictions': array([9.882026], dtype=float32)}
{'predictions': array([9.882175], dtype=float32)}
{'predictions': array([9.876069], dtype=float32)}
{'predictions': array([9.879631], dtype=float32)}
{'predictions': array([9.934914], dty

{'predictions': array([9.878841], dtype=float32)}
{'predictions': array([9.876225], dtype=float32)}
{'predictions': array([9.989506], dtype=float32)}
{'predictions': array([9.878576], dtype=float32)}
{'predictions': array([9.881716], dtype=float32)}
{'predictions': array([9.88099], dtype=float32)}
{'predictions': array([9.935731], dtype=float32)}
{'predictions': array([9.989299], dtype=float32)}
{'predictions': array([9.880482], dtype=float32)}
{'predictions': array([10.045077], dtype=float32)}
{'predictions': array([9.933466], dtype=float32)}
{'predictions': array([9.880696], dtype=float32)}
{'predictions': array([9.877882], dtype=float32)}
{'predictions': array([9.867891], dtype=float32)}
{'predictions': array([9.879485], dtype=float32)}
{'predictions': array([9.935902], dtype=float32)}
{'predictions': array([9.88072], dtype=float32)}
{'predictions': array([10.096978], dtype=float32)}
{'predictions': array([9.880935], dtype=float32)}
{'predictions': array([9.881689], dtype=float32)}


{'predictions': array([9.880612], dtype=float32)}
{'predictions': array([9.8798], dtype=float32)}
{'predictions': array([9.882003], dtype=float32)}
{'predictions': array([9.935542], dtype=float32)}
{'predictions': array([9.8815155], dtype=float32)}
{'predictions': array([9.881861], dtype=float32)}
{'predictions': array([10.095172], dtype=float32)}
{'predictions': array([9.934834], dtype=float32)}
{'predictions': array([10.09914], dtype=float32)}
{'predictions': array([9.879939], dtype=float32)}
{'predictions': array([9.991494], dtype=float32)}
{'predictions': array([9.880456], dtype=float32)}
{'predictions': array([10.096978], dtype=float32)}
{'predictions': array([9.880853], dtype=float32)}
{'predictions': array([9.883671], dtype=float32)}
{'predictions': array([9.880705], dtype=float32)}
{'predictions': array([9.878727], dtype=float32)}
{'predictions': array([9.877827], dtype=float32)}
{'predictions': array([9.879674], dtype=float32)}
{'predictions': array([9.880296], dtype=float32)}

{'predictions': array([9.936487], dtype=float32)}
{'predictions': array([9.879922], dtype=float32)}
{'predictions': array([10.045143], dtype=float32)}
{'predictions': array([9.88125], dtype=float32)}
{'predictions': array([9.8814535], dtype=float32)}
{'predictions': array([9.880743], dtype=float32)}
{'predictions': array([9.935368], dtype=float32)}
{'predictions': array([9.989063], dtype=float32)}
{'predictions': array([9.880438], dtype=float32)}
{'predictions': array([9.880066], dtype=float32)}
{'predictions': array([9.88077], dtype=float32)}
{'predictions': array([9.882027], dtype=float32)}
{'predictions': array([9.869156], dtype=float32)}
{'predictions': array([9.868244], dtype=float32)}
{'predictions': array([9.881358], dtype=float32)}
{'predictions': array([9.933754], dtype=float32)}
{'predictions': array([9.881041], dtype=float32)}
{'predictions': array([9.935447], dtype=float32)}
{'predictions': array([9.989035], dtype=float32)}
{'predictions': array([9.935098], dtype=float32)}


{'predictions': array([9.881241], dtype=float32)}
{'predictions': array([9.881105], dtype=float32)}
{'predictions': array([9.879136], dtype=float32)}
{'predictions': array([9.882427], dtype=float32)}
{'predictions': array([9.876396], dtype=float32)}
{'predictions': array([9.946648], dtype=float32)}
{'predictions': array([9.936174], dtype=float32)}
{'predictions': array([9.880049], dtype=float32)}
{'predictions': array([9.878315], dtype=float32)}
{'predictions': array([9.880494], dtype=float32)}
{'predictions': array([9.879592], dtype=float32)}
{'predictions': array([9.881511], dtype=float32)}
{'predictions': array([9.880357], dtype=float32)}
{'predictions': array([9.9894705], dtype=float32)}
{'predictions': array([9.863752], dtype=float32)}
{'predictions': array([9.880461], dtype=float32)}
{'predictions': array([9.878741], dtype=float32)}
{'predictions': array([9.871219], dtype=float32)}
{'predictions': array([9.881884], dtype=float32)}
{'predictions': array([9.881735], dtype=float32)}

{'predictions': array([9.879886], dtype=float32)}
{'predictions': array([9.879534], dtype=float32)}
{'predictions': array([9.879832], dtype=float32)}
{'predictions': array([9.880521], dtype=float32)}
{'predictions': array([9.8803215], dtype=float32)}
{'predictions': array([9.877638], dtype=float32)}
{'predictions': array([9.933733], dtype=float32)}
{'predictions': array([9.876491], dtype=float32)}
{'predictions': array([9.881297], dtype=float32)}
{'predictions': array([9.880154], dtype=float32)}
{'predictions': array([9.880337], dtype=float32)}
{'predictions': array([9.876596], dtype=float32)}
{'predictions': array([9.881325], dtype=float32)}
{'predictions': array([9.882559], dtype=float32)}
{'predictions': array([9.882852], dtype=float32)}
{'predictions': array([9.991759], dtype=float32)}
{'predictions': array([9.879984], dtype=float32)}
{'predictions': array([9.880313], dtype=float32)}
{'predictions': array([9.989496], dtype=float32)}
{'predictions': array([9.879385], dtype=float32)}

{'predictions': array([9.881144], dtype=float32)}
{'predictions': array([10.099361], dtype=float32)}
{'predictions': array([9.93506], dtype=float32)}
{'predictions': array([9.881336], dtype=float32)}
{'predictions': array([9.880962], dtype=float32)}
{'predictions': array([9.99124], dtype=float32)}
{'predictions': array([9.881255], dtype=float32)}
{'predictions': array([9.880542], dtype=float32)}
{'predictions': array([9.886691], dtype=float32)}
{'predictions': array([9.880109], dtype=float32)}
{'predictions': array([9.879889], dtype=float32)}
{'predictions': array([10.094015], dtype=float32)}
{'predictions': array([9.873494], dtype=float32)}
{'predictions': array([9.88047], dtype=float32)}
{'predictions': array([10.09782], dtype=float32)}
{'predictions': array([9.880168], dtype=float32)}
{'predictions': array([9.880856], dtype=float32)}
{'predictions': array([9.931704], dtype=float32)}
{'predictions': array([9.874772], dtype=float32)}
{'predictions': array([9.880664], dtype=float32)}
{

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [11]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 100));
print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_master': '', '_keep_checkpoint_max': 5, '_session_config': None, '_is_chief': True, '_service': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff9f81b1710>, '_global_id_in_cluster': 0, '_train_distribute': None, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_task_type': 'worker', '_task_id': 0, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_evaluation_master': '', '_log_step_count_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 96816.195, st

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [12]:
import google.datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) * 1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) * 1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) = {1}".format(base_query, phase)
  else:
    query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), {1}) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).execute().result().to_dataframe()

In [ ]:
print_rmse(model, df)

RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License